In [2]:
# 📦 載入必要套件
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, os, glob
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

# 📁 設定下載資料夾
folder = r"C:\Users\USER\Desktop\ispan\scritpts\results"
os.makedirs(folder, exist_ok=True)

# 🌿 定義要下載的蔬菜（名稱: 代碼）
vegetables = {
    "水蓮": "455",
    # 可以在這裡加入更多蔬菜
    # "韭菜": "代碼",
    # "芥藍": "代碼",
}

def download_vegetable(veg_name, veg_code, folder):
    """下載單一蔬菜資料"""
    
    # 🌐 設定瀏覽器選項
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
        "download.default_directory": folder,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    
    # 清理舊檔案（避免抓到之前的檔案）
    old_files = set(glob.glob(os.path.join(folder, "*.xls")))
    
    driver = webdriver.Chrome(options=options)
    
    try:
        print(f"\n🌿 正在下載：{veg_name} (代碼: {veg_code})")
        driver.get("https://agr.afa.gov.tw/afa/pgcropsigqty_cond.jsp")
        
        # 等待頁面載入
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "accountingyear"))
        )
        
        # 選擇選項
        Select(driver.find_element(By.NAME, "accountingyear")).select_by_value("113")
        time.sleep(0.5)
        Select(driver.find_element(By.NAME, "item")).select_by_value("03")
        time.sleep(0.5)
        Select(driver.find_element(By.NAME, "corn001")).select_by_value("02")
        time.sleep(1)
        Select(driver.find_element(By.NAME, "crop")).select_by_value(veg_code)
        time.sleep(1)
        
        # 點擊下載
        print("⬇️ 點擊下載按鈕...")
        driver.find_element(By.XPATH, '//input[@value="下  載"]').click()
        
        # 等待新檔案出現
        max_wait = 30
        start_time = time.time()
        new_file = None
        
        while time.time() - start_time < max_wait:
            current_files = set(glob.glob(os.path.join(folder, "*.xls")))
            new_files = current_files - old_files
            
            if new_files:
                new_file = list(new_files)[0]
                print(f"✅ 下載完成：{os.path.basename(new_file)}")
                break
            
            time.sleep(1)
        
        return new_file
        
    except Exception as e:
        print(f"❌ 下載時發生錯誤：{e}")
        return None
        
    finally:
        driver.quit()
        print("🔒 瀏覽器已關閉")

def process_xls_file(xls_path, output_name, folder):
    """處理下載的 XLS 檔案並轉成 CSV，然後刪除 XLS"""
    
    try:
        # 讀取 HTML 格式的 XLS
        with open(xls_path, "r", encoding="utf-8") as f:
            html = f.read()
        
        # 使用 StringIO 避免警告
        tables = pd.read_html(StringIO(html), flavor="lxml")
        
        print(f"📊 找到 {len(tables)} 個表格")
        
        # 取得第三個表格並整理
        df = tables[2]
        df.columns = df.iloc[0] + "(" + df.iloc[1] + ")"
        df = df.drop([0, 1]).reset_index(drop=True)
        
        # 儲存成 CSV
        csv_path = os.path.join(folder, f"{output_name}.csv")
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        
        print(f"📊 資料筆數：{len(df)} 筆")
        print(f"💾 已儲存：{output_name}.csv")
        
        # 🗑️ 刪除 XLS 檔案
        os.remove(xls_path)
        print(f"🗑️ 已刪除臨時檔案：{os.path.basename(xls_path)}")
        
        return df
        
    except Exception as e:
        print(f"❌ 處理檔案時發生錯誤：{e}")
        return None

# 🚀 主程式：批次下載所有蔬菜
print("=" * 60)
print("🌱 開始批次下載蔬菜資料")
print("=" * 60)

for veg_name, veg_code in vegetables.items():
    # 下載檔案
    xls_path = download_vegetable(veg_name, veg_code, folder)
    
    if xls_path:
        # 處理並轉換檔案
        df = process_xls_file(xls_path, veg_name, folder)
        
        if df is not None:
            print(f"\n📋 {veg_name} 資料預覽：")
            print(df.head())
            print("-" * 60)
    else:
        print(f"⚠️ {veg_name} 下載失敗")
    
    time.sleep(2)  # 避免請求過快

print("\n" + "=" * 60)
print("✅ 全部完成！")
print(f"📂 檔案儲存位置：{folder}")
print("=" * 60)

# 🚀 自動開啟資料夾
import subprocess
subprocess.run(['explorer', folder])

🌱 開始批次下載蔬菜資料

🌿 正在下載：水蓮 (代碼: 455)
⬇️ 點擊下載按鈕...
✅ 下載完成：pgcropsigqty_b1a8bed0cc502f43_-63454666_1997e96b989_-7f88.xls
🔒 瀏覽器已關閉
📊 找到 3 個表格
📊 資料筆數：13 筆
💾 已儲存：水蓮.csv
🗑️ 已刪除臨時檔案：pgcropsigqty_b1a8bed0cc502f43_-63454666_1997e96b989_-7f88.xls

📋 水蓮 資料預覽：
  縣市鄉鎮名稱(縣市鄉鎮名稱) 種植面積(公頃) 收穫面積(公頃) 每公頃收量(公斤)    收量(公斤)
0            高雄市   639.80   639.80     25358  16224254
1            彰化縣    22.47    22.47     22876    514028
2            花蓮縣    32.90    32.90     10285    338375
3            台北市    13.69    13.69      9976    136568
4            桃園市     1.00     1.00     17874     17874
------------------------------------------------------------

✅ 全部完成！
📂 檔案儲存位置：C:\Users\USER\Desktop\ispan\scritpts\results


CompletedProcess(args=['explorer', 'C:\\Users\\USER\\Desktop\\ispan\\scritpts\\results'], returncode=1)